In [1]:
import sys
import os

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# add parent directory to path\n",
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from paper_fetcher.country import CountryRecoder
from paper_fetcher.utils import recover_columns_names

In [2]:
FOLDER = '/home/luis/data/PubMedPapers/'

df_nosemi = pd.read_csv(
    os.path.join(FOLDER, 'publications_filtered_v7_no_semicolons.csv')
)
df_nosemi['Affiliations']=df_nosemi['Affiliations'].astype(str)
df_nosemi['Country'] = ''

df_nosemi

,Affiliations,PubMed ID,Author,Country
0,"Department of Geriatrics, Institute of Geronto...",38853292,Yucong Zhang,
1,"Key Laboratory of Vascular Aging, Ministry of ...",38853292,Yucong Zhang,
2,"Department of Geriatrics, Institute of Geronto...",38853292,Man Liu,
3,"Key Laboratory of Vascular Aging, Ministry of ...",38853292,Man Liu,
4,"Division of Cardiothoracic Vascular Surgery, T...",38853292,Jiajun Li,
...,...,...,...,...
178401,"St George's Hospital Medical School, St George...",12848262,Shirley V Hodgson,
178402,"The Lancet, 32 Jamestown Road, NW1 7BY, London...",12767753,Virginia Barbour,
178403,"Cellbank , Department of Genetics Mutagenesis,...",12693011,Tohru Masui,
178404,"MRC Human Genetics Unit, Western General Hospi...",12049178,A F Wright,


In [3]:
usa_locus = [
    "USA", "United States", "US", "Alabama", "California", "Albuquerque", "Ames IA", "Amherst MA", "Jackson MS", "Indianapolis",
    "Jacksonville FL", "Ann Arbor", "Albert Einstein College", "American Cancer Society", "Atlanta", "Massachusetts", "Mass.",
    "Boston", "Medford MA", "Milwaukee WI", "Arizona", "Baltimore", "Berkeley CA", "Bethesda MD", "Birmingham AL", "UC Davis",
    "Boulder CO", "Aurora CO", "Cambridge MA", "Harvard MA", "Brookline MA", "Brownsville TX", "Bronx NY", "New Haven CT", "Evanston, IL", "Evanston, Ill",
    "New Brunswick NJ", "Buffalo NY", "Burlington VT", "Califirnia", "New York", "Norwood MA", "Oakland", "Oregon", "Mountain View",
    "Palo Alto CA", "San Francisco CA", "Chapel Hill NC", "Charleston SC", "Charlestown MA", "Charlotte NC", "Dallas", "Rockville, MD",
    "Pennsylvania PA", "Pennsylvania", "Phoenix AZ", "Charlottesville VA", "Chicago", "Cincinnati", "Claremont CA", "Bethesda, Md",
    "Cleveland OH", "Collegeville PA", "Columbia SC", "Columbia University", "Providence RI", "Chevy Chase MD", "Bethesda, MD",
    "Puerto Rico", "Redwood City CA", "Triangle Park NC", "Columbus OH", "Corona CA", "Philadelphia", "Rochester MN", "Rochester, MN",
    "Rockville MD", "Sacramento CA", "Salt Lake City", "San Antonio TX", "Seattle", "Rhode Island", "Brooklyn College", "Portland, ME",
    "Brigham and Womens Hospital", "Calico", "Sioux Center IA", "CedarsSinai", "Los Angeles", "Colorado", "Danville PA", "San Diego, CA",
    "Davis CA", "Decatur GA", "Denver", "Stanford University", "Stanford Medical School", "Delaware", "West Virginia", "Mayo Clinic",
    "Louisville KY", "Brown University", "Blacksburg VA", "District of Columbia", "Downers Grove IL", "Duarte CA", "Indiana University",
    "Duke University", "Tennessee", "Texas", "East Lansing", "Emory University", "Englewood CO", "Evanston IL", "Rochester, Minn.",
    "George Washington University", "Icahn School", "Durham NC", "Farmington CT", "Florida", "Fort Collins CO", "Saint Louis University",
    "Fort Worth TX", "Framingham", "Frederick MD", "Fremont CA", "Gainesville", "Gaithersburg MD", "Grand Rapids MI", 
    "Glen Echo MD", "Hagerstown MD", "Hanover NH", "Harrison NJ", "Hartford CT", "Hawaii", "Mississippi", "Michigan", "San Francisco VA",
    "Miami", "Utah", "Honolulu", "Houston", "Illinois", "Iowa", "Irvine CA", "Ithaca NY", "Johns Hopkins", "Kalamazoo MI",
    "Kansas", "Kentucky", "Prussia PA", "La Jolla", "Lancaster PA", "Las Vegas NV", "Lewisburg PA", "Lexington KY", " Murdoch Children's Research Institute ",
    "Lisle IL", "Pittsburgh PA", "Yale Universit", "Yale School of Medicine", "Wisconsin", "Winston Salem NC", " Alvin J. Siteman Cancer Center",
    "WinstonSalem NC", "Washington", "Waco TX", "Virginia", "Vermont", "Vanderbilt", "Maryland", "North Carolina", "James J. Peters Veterans Affairs",
    "Pittsburgh", "Tulane", "Tucson", "Torrance", "Wake Forest School of Medicine", "VA Palo Alto", "VA San Diego", "National Institute on Aging",
    "Tarrytown NY", "Long Beach CA", "Louisiana", "Madison WI", "Masschusetts", "Memphis TN", "Charles Bronfman Institute", "Hawai'i",
    "Harvard", "Lerner Research Institute", "Minneapolis", "Minnesota", "Missouri", "Moffitt Cancer Center", "New Haven, CT",
    "Morristown NJ", "Mountain View CA", "Nashville", "Marina del Rey", "Maywood IL", "South Carolina", "Tampa FL", "Providence, RI",
    "Monell Chemical Senses Center", "Blood Institute MA", "Taussig Cancer Institute", "Tallahassee", "Syracuse NY", "East Carolina University",
    "Redwood City", "Oklahoma", "Ohio", "Palo Alto", "Portland OR", "New Hyde Park NY", "New Hampshire", "New Orleans", "Hagerstown, MD", "Medford, MA",
    "New Jersey", "North Haven CT", "Northeastern University", "UAB Lung Imaging Lab", "Stony Brook NY", "Stamford CT", " Dana-Farber Cancer Institute",
    "Stanford", "St Louis MO", "Connecticut", "ChanZuckerberg Biohub", "Cleveland", "Greenville NC", "San Diego CA", "Brigham Women's Hospital", "Ithaca, NY",
    "Demoulas Center for Cardiac Arrhythmias", "Portland ME", "SUNY Downstate College", "SWOG Statistical Center", "Broad Institute", "Norwood, MA", "Aurora, CO",
    "Santa Cruz CA", "Scottsdale AZ", "Scripps", "Severn Health Solutions", "Severna Park MD", "Frank H. Netter MD School of Medicine", "Memphis, TN",
    "NIH ENIGMA Center for Worldwide Medicine", "San Francisco, CA", "DAnville, PA", "Charlotte, NC", "Las Vegas", "San Antonio, TX", "Stony Brook, NY",
    "Winston-Salem, NC", "Frederick, MD", "Durham, NC", "Waco, TX", "Pleasanton, CA", "Danville, PA", "Corona, CA", "Farmington, CT", "Lewisburg, PA", "Lancaster, PA",
    "Phoenix, AZ", "Glen Echo, MD", "Tarrytown, NY", "Cambridge, MA", "Sioux Center, IA", "University of Chapel Hill", "Long Beach, CA", "Collegeville, PA",
    "Gaithersburg, MD", "Syracuse, NY", "New Brunswick, NJ", "Buffalo, NY", "King of Prussia, PA", "Portland, OR", "Boulder, CO", "Grand Rapids, MI",
    "Claremont, CA", "Brookline, MA", "Duarte, CA", "Morristown, NJ", "Kalamazoo, MI", "Cambridge, Mass", "Chevy Chase, MD", "Lisle, IL", "Harrison, NJ",
    "Dartmouth College", "New Hyde Park, NY", "Columbus, OH", "Jackson, MS", "Tucker, Georgia", "Downers Grove, IL", "Triangle Park, NC", "Novato", "Penn-CHOP",
    "Herman B Wells Center for Pediatric Research", "Chan-Zuckerberg Biohub", "Eden Prairie", "Robert D. Patricia E. Kern Center for Science of Health Care Delivery",
    "Stamford, CT"
]

In [4]:
alb_locus = ["Albania", "Tirana"]
arg_locus = ["Argentina", "Buenos Aires"]
arm_locus = ["Armenia"]
aus_locus = ["Australia", "Melbourne", "Perth", "Cancer Council Victoria", "Curtin University", "Sydney", "Queensland", "Monash Health", "Tasmania",
             "Monash University", "Australi", "Royal Victorian Eye and Ear Hospital", "University of New South Wales", "Peninsula Health",
             "National Centre for Healthy Ageing", "NHMRC", "New South Wales", "Darlinghurst"]
aut_locus = ["Austria", "Graz", "Innsbruck"]

bah_locus = ["Bahrain"]
ban_locus = ["Bangladesh", "Dhaka"]
bar_locus = ["Barbados"]
bel_locus = ["Belgium", "Leuven"]
ben_locus = ["Benin", "Bénin"]
blr_locus = ["Belarus"]
bol_locus = ["Bolivia"]
bos_locus = ["Bosnia and Herzegovina", "Sarajevo", "Bosnia", "Bosnia Herzegovina"]
bot_locus = ["Botswana"]
bra_locus = ["Brazil", "Brasil"]
bru_locus = ["Brunei"]
bul_locus = ["Bulgaria", "Sofia"]
bur_locus = ["Burkina Faso", "Ouagadougou"]

cam_locus = ["Cameroon", "Cameroun"]
cmb_locus = ["Cambodia", "Phnom Penh"]
can_locus = ["Canada", "British Columbia", "Ontario", "Québec", "Quebec", "Toronto", "Vancouver", "Lady Davis Institute", "Saskatchewan",
             "Montreal", "Hamilton Health Sciences", "McGill University", "Sunnybrook Research Institute", "Thrombosis Atherosclerosis Research Institute",
             "Sir Mortimer B. Davis-Jewish General Hospital"]
chi_locus = ["Chile", "Talca"]
chn_locus = ["China", "Beijing", "CHINA", "Chinese", "Central South University", "Changping", "Chengdu", "Dongfang",
             "Xian Jiaotong", "Tongxiang", "Fudan University", "Guangzhou", "Guangdong", "Harbin", "Heilongjiang", "Sun Yat-sen University",
             "Furong", "Guangxi", "Hong Kong", "Hongqiao", "Dongguan", "Hunan", "Jiangsu", "Jinan", "Zhengzhou", "NHC Key Laboratory of Cardiovascular Molecular Biology",
             "Zhongshan", "Liaoning", "Liuzhou", "Zhejiang", "Yichang", "Xiangya", "Wuhan", "Tianjin", "Tsinghua", "Jiaotong",
             "Wannan Medical College", "Nanchang", "Shandong", "Li Ka Shing Faculty of Medicine", "Xuanwu", "Tongji", "Suzhou",
             "Soochow University", "Oujiang", "Peking", "Pengzhou", "Yueyang", "Yinchuan", "Yantai", "Fujian", "Key Laboratory of Molecular Cardiovascular Sciences",
             "Sun Yatsen University", "Shiyan", "Shidong", "Shenzhen", "Shengjing", "Guangan", "Huanggang", "Hubei", "Emergency Medicine Difficult Diseases Institute",
             "Huashan", "Nanjing", "Shaanxi", "Shanghai", "Southern Medical University", " State Key Laboratory of Organ Failure Research",
             "National Clinical Research Center for Kidney Disease", "Department of Neurology, Second Affiliated Hospital"]
civ_locus = ["Côte dIvoire", "Côte d'Ivoire"]
col_locus = ["Colombia", "Bogotá", "Bogota"]
con_locus = ["Brazzaville"]
cro_locus = ["Croatia"]
cub_locus = ["Cuba", "Havana", "Matanzas", "Habana"]
crc_locus = ["Costa Rica"]
cyp_locus = ["Cyprus"]
cz_locus  = ["Czech", "Prague"]

den_locus = ["Denmark", "Aarhus", "Danish", "Danmark", "Faroe Islands", "Greenland", "Herlev and Gentofte Hospital",
             "Rigshospitalet", "Copenhagen"]
dom_locus = ["Dominican Republic", "Dominicana"]
drc_locus = ["Democratic Republic of Congo", "Kinshasa", "DR Congo", "University of Kisangani"]

ecu_locus = ["Ecuador", "Quito", "Guayaquil"]
egy_locus = ["Egypt"]
eri_locus = ["Eritrea"]
esp_locus = ["Spain", "España",  "Barcelona", "Madrid", "Ramón y Cajal", "Asturias", "Pamplona", "Granada", "BCNAIM", "Alberto Sols", "IRYCIS",
             "GRINCAVA", "Murcia", "Oviedo", "Bellvitge Biomedical Research Institute", "CIBERESP", "Gipuzkua", "Sevilla", "Gipuzkoa", "Barakaldo",
             "Alcalá de Henares", "Hospital Universitario La Paz", "Basque"]
est_locus = ["Estonia", "Tartu"]
eth_locus = ["Ethiopia"]

fij_locus = ["Fiji"]
fin_locus = ["Finland", "Oulu", "FINLAND", "Finlan", "Helsinki", "Kuopio", "Finnish", "Pirkanmaa", "University of Tampere"]
fra_locus = ["France", "Marseille", "Bordeaux", "Paris", "Inserm", "Centre de Recherche des Cordeliers", "Créteil", " Laboratoire d'Excellence GR-Ex",
             "Dijon", "Lille", "Nantes", "Montpellier", "la Santé", "Lyon", "Villejuif", "CNRS", "Toulouse", " CHRU Nancy", "INSERM"]

gab_locus = ["Gabon"]
gam_locus = ["Gambia"]
gbi_locus = ["Guinea-Bissau"]
geo_locus = ["Tbilisi"]
ger_locus = ["Germany", "Heidelberg", "Augsburg", "Berlin", "Munich", "Deutschland", "Essen", "Freiburg", "Fraunhofer", "Leipzig",
             "German", "Greifswald", "Helmholtz", "Erlangen", "Hamburg", "DKFZ", "DZHK", "LIFE Child", "MaxPlanck",
             "Neuherberg", "CharitéUniversität", "FriedrichAlexanderUniversität", "HumboldtUniversität", "Max-Planck Institute",
             "LudwigMaximilian", "Köln", "Cologne"]
gha_locus = ["Ghana"]
gre_locus = ["Greece", "University of Crete", "University of Athens"]

hun_locus = ["Hungary", "Budapest"]

ice_locus = ["Iceland", "Kopavogur"]
ind_locus = ["Kolkata", "Delhi", "Punjab", "Karnataka", "Hyderabad", "Pune", "Udhampur", "Haryana", "Kerala", "Pondicherry", "Ghaziabad", "Chennai", "Lucknow",
             "Kashmir", "Mumbai", "Ranchi", "Ludhiana", "Mysore", "Bengaluru", "Kochi", "Jaipur", "Bangalore", "Pradesh", "Chandigarh", "Manipur", "Bhopal", "Aligarh",
             "Secunderabad", "Mysuru", "Manipal", "Coimbatore", "Gujarat", "Orissa", "Rampur", "Tamil Nadu"]
ina_locus = ["Indonesia", "Jakarta"]
ire_locus = ["Ireland", "Dublin"]
irn_locus = ["Iran"]
irq_locus = ["Iraq"]
isr_locus = ["Israel"]
ita_locus = ["Italy", "Ragusa","Sassari", "Rome", "Bergamo", "Roma", "DIBRIS", "Torino", "Florence", "Fondazione IRCCS", "Humanitas University",
             "ITALY", "Italia", "Itlay", "Milan", "Padua", "Pavia", "Università Cattolica Sacro Cuore", "Udine", "Padova", "Catanzaro", "Federico II University"]

jam_locus = ["Jamaica"]
jor_locus = ["Jordan", "Amman"]
jpn_locus = ["Japan", "Tokyo", "Osaka", "Chiba University", "Tokushima", "Tohoku", "Fukuoka", "Hiroshima",
             "Iwate Medical University", "Kanagawa", "Nagoya", "Gifu", "Otsu", "RIKEN Center", "Saga University"]

kaz_locus = ["Kazakhstan"]
ken_locus = ["Kenya", "Nairobi"]
kos_locus = ["Kosovo", "Pristina", "Prishtina"]
ksa_locus = ["Saudi Arabia", "Riyadh", "Jazan"]
kuw_locus = ["Kuwait"]

lao_locus = ["Laos"]
lat_locus = ["Latvia", "Riga"]
leb_locus = ["Lebanon", "Beirut"]
lie_locus = ["Liechtenstein"]
lth_locus = ["Lithuania"]
lux_locus = ["Luxembourg"]

mac_locus = ["Macedonia", "Skopje"]
mad_locus = ["Madagascar"]
mal_locus = ["Mali"]
mat_locus = ["Malta"]
mau_locus = ["Mauritius"]
mex_locus = ["Mexico", "México"]
mlw_locus = ["Malawi"]
mly_locus = ["Malaysia"]
mor_locus = ["Morocco", "Rabat"]
mzb_locus = ["Mozambique"]

nam_locus = ["Namibia", "Windhoek"]
nep_locus = ["Nepal"]
nig_locus = ["Nigeria", "Abuja", "Lagos"]
nor_locus = ["Norway", "Oslo", "Bergen", "Norwegian", "Trondheim"]
nth_locus = ["Netherlands", "Amsterdam", "EMGO", "Vrije Universiteit", "Leiden", "Maastricht", "Utrecht", "Hubrecht", "Sanquin Diagnostic Services",
             "GRIAC", "Julius Center for Health Sciences", "VU University Medical Center", "Rotterdam", "Nederland",
             "Bilthoven", "Radboud", "Delft University", "Rijnstate", "Julius Center"]
nz_locus  = ["New Zealand", "Auckland", "University of Otago"]

oma_locus = ["Oman"]

pak_locus = ["Pakistan"]
pan_locus = ["Panama", "Panamá"]
per_locus = ["Perú", "Peru"]
phi_locus = ["Philippines", "Manila"]
png_locus = ["Papua New Guinea"]
pol_locus = ["Poland", "Warsaw", "Lódz"]
por_locus = ["Portugal", "Aveiro", "Coimbra"]

qat_locus = ["Qatar", "Doha"]

rom_locus = ["Romania", "Ana Aslan International Foundation"]
rsa_locus = ["South Africa", "University of the Witwatersrand", "Cape Town", "University of Witwatersrand"]
rus_locus = ["Russia", "Moscow"]
rwa_locus = ["Rwanda"]

sam_locus = ["Samoa", "Apia"]
sen_locus = ["Senegal", "Dakar"]
sey_locus = ["Seychelles"]
sin_locus = ["Singapore", "Nanyang"]
skr_locus = ["South Korea", "Republic of Korea", "Busan", "Seoul", "Cheongjusi", "Chungnam", "Daegu", "Daejeon", "Goyangsi", "Cheongju",
             "Gwangju", "Gyeonggido", "Ulsan", "Hwasun", "Korea University", "Uijeongbu Eulji", "Yonsei", "South of Korea", "Yongin", "Wonju",
             "Seongnam", "Gyeonggi-do", "Samsung Medical Center", "Cheongju-si", "Jeonju", "Suwon", "Goyang", "Cheongju", "Chuncheon", "Chungcheongbuk-do",
             "Korea Mouse Phenotyping"]
sle_locus = ["Sierra Leone"]
slk_locus = ["Slovak"]
slv_locus = ["Slovenia"]
sri_locus = ["Sri Lanka"]
srb_locus = ["Serbia"]
sud_locus = ["Sudan", "Khartoum"]
swe_locus = ["Sweden", "Stockholm", "Malmö", "Lund", "Umeå", "Karolinska Institute", "Linköping", "Uppsala", "Sweded",
             "Karolinska"]
swi_locus = ["Switzerland", "Botnar Research Centre", "Zurich", "Bern", "Geneva", "Swiss", "Lausanne"]

tai_locus = ["Taiwan", "Taipei", "Chung Shan Medical", "Taichung", "Hualien", "Kaohsiung", "Taoyuan", "Tamkang", "Hsinchu",
             "National YangMing University", "Yang-Ming University", "Tri-Service General Hospital"]
taj_locus = ["Tajikistan"]
tan_locus = ["Tanzania", "Dar es Salaam"]
tha_locus = ["Thailand"]
tun_locus = ["Tunisia", "Sousse"]
tur_locus = ["Turkey"]

uae_locus = ["UAE", "United Arab Emirates", "Abu Dhabi"]
uga_locus = ["Uganda"]
uk_locus  = ["United Kingdom", "UK", "Aberdeen", "Addenbrookes Hospital", "N Ireland", "Scotland", "Belfast", "Bristol", "Novo Nordisk Research Centre",
            "Cardiff", "London", "NHS Trust", "NHS trust", "British", "Edinburgh", "Glasgow", "NHS Foundation Trust", "National Problem Gambling Clinic",
            "John Radcliffe Hospital", "Lancaster University", "Leeds", "Leicester", "Liverpool", "United ingdom", "Oxford Centre for Computational Neuroscience",
            "United kingdom", "Manchester", "UNITED KINGDOM", "Wellcome Centre for Human Genetics", "Lothian", "University of Warwick", "U.K",
            "Wellcome Trust Centre for Human Genetics", "Loughborough University", "Ormond Street Hospital", "NIHR", "Alderley Park", "NHS Dumfries Galloway",
            "MRC Integrative Epidemiology", "MRC Population Health Research", "MRCBHF", "MRCPHE", "University of Exeter", "University of Nottingham",
            "Medical Research Council", "BHF Data Science Centre", "BRC Oxford Haematology", "University of Southampton", "Centre for Drug Safety Science",
            "Francis Crick Institute", "Robertson Center for Biostatistics", "Midlothian", "Worcestershire", "Newcastle University", "Middlesex",
            "UCL Institute of Child Health", "Swansea University", "Gibraltar", "Nuffield", "University of Cambridge", "Royal National Hospital for Rheumatic Diseases",
            "University of Oxford", "U.K.", "UCL Queen Square Institute of Neurology", "University of Birmingham", "Mount Vernon Hospital", "Norwich", "Oxford, England",
            "Addenbrooke's Hospital", "Southampton, Hampshire", "Radcliffe Department of Medicine", "BHF Cardiovascular Epidemiology Unit"]
ukr_locus = ["Ukraine"]
uru_locus = ["Uruguay", "Montevideo"]
uzb_locus = ["Uzbekistan"]

ven_locus = ["Venezuela", "Caracas"]
vie_locus = ["Vietnam", "Hanoi"]

zam_locus = ["Zambia"]
zim_locus = ["Zimbabwe"]

In [5]:
europe_pipe = Pipeline([
    ('alb', CountryRecoder(country_name="Albania", aliases=alb_locus).set_output(transform='pandas')),
    ('uk', CountryRecoder(country_name="UK", aliases=uk_locus).set_output(transform='pandas')),
    ('ire', CountryRecoder(country_name="Ireland", aliases=ire_locus).set_output(transform='pandas')),
    ('gre', CountryRecoder(country_name="Greece", aliases=gre_locus).set_output(transform='pandas')),
    ('esp', CountryRecoder(country_name="Spain", aliases=esp_locus).set_output(transform='pandas')),
    ('por', CountryRecoder(country_name="Portugal", aliases=por_locus).set_output(transform='pandas')),
    ('fra', CountryRecoder(country_name="France", aliases=fra_locus).set_output(transform='pandas')),
    ('bel', CountryRecoder(country_name="Belgium", aliases=bel_locus).set_output(transform='pandas')),
    ('ger', CountryRecoder(country_name="Germany", aliases=ger_locus).set_output(transform='pandas')),
    ('aut', CountryRecoder(country_name="Austria", aliases=aut_locus).set_output(transform='pandas')),
    ('ita', CountryRecoder(country_name="Italy", aliases=ita_locus).set_output(transform='pandas')),
    ('net', CountryRecoder(country_name="Netherlands", aliases=nth_locus).set_output(transform='pandas')),
    ('pol', CountryRecoder(country_name="Poland", aliases=pol_locus).set_output(transform='pandas')),
    ('swi', CountryRecoder(country_name="Switzerland", aliases=swi_locus).set_output(transform='pandas')),
    ('swe', CountryRecoder(country_name="Sweden", aliases=swe_locus).set_output(transform='pandas')),
    ('nor', CountryRecoder(country_name="Norway", aliases=nor_locus).set_output(transform='pandas')),
    ('den', CountryRecoder(country_name="Denmark", aliases=den_locus).set_output(transform='pandas')),
    ('fin', CountryRecoder(country_name="Finland", aliases=fin_locus).set_output(transform='pandas')),
    ('ice', CountryRecoder(country_name="Iceland", aliases=ice_locus).set_output(transform='pandas')),
    ('hun', CountryRecoder(country_name="Hungary", aliases=hun_locus).set_output(transform='pandas')),
    ('rus', CountryRecoder(country_name="Russia", aliases=rus_locus).set_output(transform='pandas')),
    ('cz', CountryRecoder(country_name="Czech Republic", aliases=cz_locus).set_output(transform='pandas')),
    ('blr', CountryRecoder(country_name='Belarus', aliases=blr_locus).set_output(transform='pandas')),
    ('est', CountryRecoder(country_name='Estonia', aliases=est_locus).set_output(transform='pandas')),
    ('ukr', CountryRecoder(country_name='Ukraine', aliases=ukr_locus).set_output(transform='pandas')),
    ('rom', CountryRecoder(country_name='Romania', aliases=rom_locus).set_output(transform='pandas')),
    ('slk', CountryRecoder(country_name='Slovakia', aliases=slk_locus).set_output(transform='pandas')),
    ('slv', CountryRecoder(country_name='Slovenia', aliases=slv_locus).set_output(transform='pandas')),
    ('cro', CountryRecoder(country_name='Croatia', aliases=cro_locus).set_output(transform='pandas')),
    ('srb', CountryRecoder(country_name='Serbia', aliases=srb_locus).set_output(transform='pandas')),
    ('lth', CountryRecoder(country_name='Lithuania', aliases=lth_locus).set_output(transform='pandas')),
    ('lux', CountryRecoder(country_name='Luxembourg', aliases=lux_locus).set_output(transform='pandas')),
    ('lie', CountryRecoder(country_name='Liechtenstein', aliases=lie_locus).set_output(transform='pandas')),
    ('bos', CountryRecoder(country_name='Bosnia and Herzegovina', aliases=bos_locus).set_output(transform='pandas')),
    ('mac', CountryRecoder(country_name='Macedonia', aliases=mac_locus).set_output(transform='pandas')),
    ('mat', CountryRecoder(country_name='Malta', aliases=mat_locus).set_output(transform='pandas')),
    ('geo', CountryRecoder(country_name='Georgia', aliases=geo_locus).set_output(transform='pandas')),
    ('kos', CountryRecoder(country_name='Kosovo', aliases=kos_locus).set_output(transform='pandas')),
    ('arm', CountryRecoder(country_name='Armenia', aliases=arm_locus).set_output(transform='pandas')),
    ('bul', CountryRecoder(country_name='Bulgaria', aliases=bul_locus).set_output(transform='pandas')),
    ('lat', CountryRecoder(country_name='Latvia', aliases=lat_locus).set_output(transform='pandas')),
])

europe_trans = ColumnTransformer([
    ('europe', europe_pipe, ['Affiliations', 'Country'])
], remainder='passthrough').set_output(transform='pandas')



In [6]:
df_europe = europe_trans.fit_transform(df_nosemi)
df_europe.columns = recover_columns_names(df_europe)

In [7]:
america_pipe = Pipeline([
    ('usa', CountryRecoder(country_name="USA", aliases=usa_locus).set_output(transform='pandas')),
    ('bra', CountryRecoder(country_name="Brazil", aliases=bra_locus).set_output(transform='pandas')),
    ('can', CountryRecoder(country_name="Canada", aliases=can_locus).set_output(transform='pandas')),
    ('chi', CountryRecoder(country_name="Chile", aliases=chi_locus).set_output(transform='pandas')),
    ('col', CountryRecoder(country_name='Colombia', aliases=col_locus).set_output(transform='pandas')),
    ('mex', CountryRecoder(country_name='Mexico', aliases=mex_locus).set_output(transform='pandas')),
    ('per', CountryRecoder(country_name='Peru', aliases=per_locus).set_output(transform='pandas')),
    ('jam', CountryRecoder(country_name='Jamaica', aliases=jam_locus).set_output(transform='pandas')),
    ('dom', CountryRecoder(country_name='Dominican Republic', aliases=dom_locus).set_output(transform='pandas')),
    ('pan', CountryRecoder(country_name='Panama', aliases=pan_locus).set_output(transform='pandas')),
    ('arg', CountryRecoder(country_name='Argentina', aliases=arg_locus).set_output(transform='pandas')),
    ('uru', CountryRecoder(country_name='Uruguay', aliases=uru_locus).set_output(transform='pandas')),
    ('bar', CountryRecoder(country_name='Barbados', aliases=bar_locus).set_output(transform='pandas')),
    ('bol', CountryRecoder(country_name='Bolivia', aliases=bol_locus).set_output(transform='pandas')),
    ('cub', CountryRecoder(country_name='Cuba', aliases=cub_locus).set_output(transform='pandas')),
    ('crc', CountryRecoder(country_name='Costa Rica', aliases=crc_locus).set_output(transform='pandas')),
    ('ven', CountryRecoder(country_name='Venezuela', aliases=ven_locus).set_output(transform='pandas')),
    ('ecu', CountryRecoder(country_name='Ecuador', aliases=ecu_locus).set_output(transform='pandas')),
])

america_trans = ColumnTransformer([
    ('america', america_pipe, ['Affiliations', 'Country'])
], remainder='passthrough').set_output(transform='pandas')

In [8]:
df_america = america_trans.fit_transform(df_europe)
df_america.columns = recover_columns_names(df_america)

In [9]:
africa_pipe = Pipeline([
    ('egy', CountryRecoder(country_name='Egypt', aliases=egy_locus).set_output(transform='pandas')),
    ('rsa', CountryRecoder(country_name='South Africa', aliases=rsa_locus).set_output(transform='pandas')),
    ('gha', CountryRecoder(country_name='Ghana', aliases=gha_locus).set_output(transform='pandas')),
    ('nig', CountryRecoder(country_name='Nigeria', aliases=nig_locus).set_output(transform='pandas')),
    ('eth', CountryRecoder(country_name='Ethiopia', aliases=eth_locus).set_output(transform='pandas')),
    ('ken', CountryRecoder(country_name='Kenya', aliases=ken_locus).set_output(transform='pandas')),
    ('nam', CountryRecoder(country_name='Namibia', aliases=nam_locus).set_output(transform='pandas')),
    ('tan', CountryRecoder(country_name='Tanzania', aliases=tan_locus).set_output(transform='pandas')),
    ('sle', CountryRecoder(country_name='Sierra Leone', aliases=sle_locus).set_output(transform='pandas')),
    ('uga', CountryRecoder(country_name='Uganda', aliases=uga_locus).set_output(transform='pandas')),
    ('cam', CountryRecoder(country_name='Cameroon', aliases=cam_locus).set_output(transform='pandas')),
    ('rwa', CountryRecoder(country_name='Rwanda', aliases=rwa_locus).set_output(transform='pandas')),
    ('zim', CountryRecoder(country_name='Zimbabwe', aliases=zim_locus).set_output(transform='pandas')),
    ('sen', CountryRecoder(country_name='Senegal', aliases=sen_locus).set_output(transform='pandas')),
    ('civ', CountryRecoder(country_name='Côte dIvoire', aliases=civ_locus).set_output(transform='pandas')),
    ('mzb', CountryRecoder(country_name='Mozambique', aliases=mzb_locus).set_output(transform='pandas')),
    ('mal', CountryRecoder(country_name='Mali', aliases=mal_locus).set_output(transform='pandas')),
    ('mlw', CountryRecoder(country_name='Malawi', aliases=mlw_locus).set_output(transform='pandas')),
    ('gam', CountryRecoder(country_name='Gambia', aliases=gam_locus).set_output(transform='pandas')),
    ('gab', CountryRecoder(country_name='Gabon', aliases=gab_locus).set_output(transform='pandas')),
    ('drc', CountryRecoder(country_name='DR Congo', aliases=drc_locus).set_output(transform='pandas')),
    ('bur', CountryRecoder(country_name='Burkina Faso', aliases=bur_locus).set_output(transform='pandas')),
    ('sud', CountryRecoder(country_name='Sudan', aliases=sud_locus).set_output(transform='pandas')),
    ('mor', CountryRecoder(country_name='Morocco', aliases=mor_locus).set_output(transform='pandas')),
    ('tun', CountryRecoder(country_name='Tunisia', aliases=tun_locus).set_output(transform='pandas')),
    ('mau', CountryRecoder(country_name='Mauritius', aliases=mau_locus).set_output(transform='pandas')),
    ('mad', CountryRecoder(country_name='Madagascar', aliases=mad_locus).set_output(transform='pandas')),
    ('con', CountryRecoder(country_name='Congo', aliases=con_locus).set_output(transform='pandas')),
    ('eri', CountryRecoder(country_name='Eritrea', aliases=eri_locus).set_output(transform='pandas')),
    ('ben', CountryRecoder(country_name='Benin', aliases=ben_locus).set_output(transform='pandas')),
    ('gbi', CountryRecoder(country_name='Guinea-Bissau', aliases=gbi_locus).set_output(transform='pandas')),
    ('sey', CountryRecoder(country_name='Seychelles', aliases=sey_locus).set_output(transform='pandas')),
    ('zam', CountryRecoder(country_name='Zambia', aliases=zam_locus).set_output(transform='pandas')),
    ('bot', CountryRecoder(country_name='Botswana', aliases=bot_locus).set_output(transform='pandas')),
])

africa_trans = ColumnTransformer([
    ('africa', africa_pipe, ['Affiliations', 'Country'])
], remainder='passthrough').set_output(transform='pandas')

In [10]:
df_africa = africa_trans.fit_transform(df_america)
df_africa.columns = recover_columns_names(df_africa)

In [11]:
asia_pipe = Pipeline([ 
    ('chn', CountryRecoder(country_name="China", aliases=chn_locus).set_output(transform='pandas')),
    ('jpn', CountryRecoder(country_name="Japan", aliases=jpn_locus).set_output(transform='pandas')),
    ('tai', CountryRecoder(country_name="Taiwan", aliases=tai_locus).set_output(transform='pandas')),
    ('sin', CountryRecoder(country_name="Singapore", aliases=sin_locus).set_output(transform='pandas')),
    ('skr', CountryRecoder(country_name="South Korea", aliases=skr_locus).set_output(transform='pandas')),
    ('cyp', CountryRecoder(country_name="Cyprus", aliases=cyp_locus).set_output(transform='pandas')),
    ('isr', CountryRecoder(country_name='Israel', aliases=isr_locus).set_output(transform='pandas')),
    ('tha', CountryRecoder(country_name='Thailand', aliases=tha_locus).set_output(transform='pandas')),
    ('ksr', CountryRecoder(country_name='Saudi Arabia', aliases=ksa_locus).set_output(transform='pandas')),
    ('pak', CountryRecoder(country_name='Pakistan', aliases=pak_locus).set_output(transform='pandas')),
    ('mly', CountryRecoder(country_name='Malaysia', aliases=mly_locus).set_output(transform='pandas')),
    ('qat', CountryRecoder(country_name='Qatar', aliases=qat_locus).set_output(transform='pandas')),
    ('uae', CountryRecoder(country_name='United Arab Emirates', aliases=uae_locus).set_output(transform='pandas')),
    ('ban', CountryRecoder(country_name='Bangladesh', aliases=ban_locus).set_output(transform='pandas')),
    ('tur', CountryRecoder(country_name='Turkey', aliases=tur_locus).set_output(transform='pandas')),
    ('irn', CountryRecoder(country_name='Iran', aliases=irn_locus).set_output(transform='pandas')),
    ('kuw', CountryRecoder(country_name='Kuwait', aliases=kuw_locus).set_output(transform='pandas')),
    ('ind', CountryRecoder(country_name='India', aliases=ind_locus).set_output(transform='pandas')),
    ('irq', CountryRecoder(country_name='Iraq', aliases=irq_locus).set_output(transform='pandas')),
    ('nep', CountryRecoder(country_name='Nepal', aliases=nep_locus).set_output(transform='pandas')),
    ('sri', CountryRecoder(country_name='Sri Lanka', aliases=sri_locus).set_output(transform='pandas')),
    ('ina', CountryRecoder(country_name='Indonesia', aliases=ina_locus).set_output(transform='pandas')),
    ('leb', CountryRecoder(country_name='Lebanon', aliases=leb_locus).set_output(transform='pandas')),
    ('phi', CountryRecoder(country_name='Philipines', aliases=phi_locus).set_output(transform='pandas')),
    ('png', CountryRecoder(country_name='Papua New Guinea', aliases=png_locus).set_output(transform='pandas')),
    ('oma', CountryRecoder(country_name='Oman', aliases=oma_locus).set_output(transform='pandas')),
    ('bru', CountryRecoder(country_name='Brunei', aliases=bru_locus).set_output(transform='pandas')),
    ('jor', CountryRecoder(country_name='Jordan', aliases=jor_locus).set_output(transform='pandas')),
    ('vie', CountryRecoder(country_name='Viet Nam', aliases=vie_locus).set_output(transform='pandas')),
    ('kaz', CountryRecoder(country_name='Kazakhstan', aliases=kaz_locus).set_output(transform='pandas')),
    ('cmb', CountryRecoder(country_name='Cambodia', aliases=cmb_locus).set_output(transform='pandas')),
    ('taj', CountryRecoder(country_name='Tajikistan', aliases=taj_locus).set_output(transform='pandas')),
    ('uzb', CountryRecoder(country_name='Uzbekistan', aliases=uzb_locus).set_output(transform='pandas')),
    ('bah', CountryRecoder(country_name='Bahrain', aliases=bah_locus).set_output(transform='pandas')),
    ('lao', CountryRecoder(country_name='Laos', aliases=lao_locus).set_output(transform='pandas')),
])

asia_trans = ColumnTransformer([
    ('country', asia_pipe, ['Affiliations', 'Country'])
], remainder='passthrough').set_output(transform='pandas')


In [12]:
df_asia = asia_trans.fit_transform(df_africa)
df_asia.columns = recover_columns_names(df_asia)

In [13]:
australasia_pipe = Pipeline([
    ('aus', CountryRecoder(country_name="Australia", aliases=aus_locus).set_output(transform='pandas')),
    ('nz', CountryRecoder(country_name="New Zealand", aliases=nz_locus).set_output(transform='pandas')),
    ('fij', CountryRecoder(country_name="Fiji", aliases=fij_locus).set_output(transform='pandas')),
    ('sam', CountryRecoder(country_name="Samoa", aliases=sam_locus).set_output(transform='pandas')),
])

australasia_trans = ColumnTransformer([
    ('australasia', australasia_pipe, ['Affiliations', 'Country'])
], remainder='passthrough').set_output(transform='pandas')

In [14]:
df_count = australasia_trans.fit_transform(df_asia)
df_count.columns = recover_columns_names(df_count)

In [15]:
nocountry = df_count[df_count['Country'] == ''].reset_index(drop=True)
nocountry = nocountry.drop(columns=['Country'], inplace=False)
nocountry = nocountry[nocountry['Affiliations'] != 'nan'].reset_index(drop=True)
nocountry

,Affiliations,PubMed ID,Author
0,Department of Epidemiology Biostatistics,38690905,Yi-Xiang Ye
1,Department of Epidemiology Biostatistics,38690905,Jun-Xiang Chen
2,Department of Epidemiology Biostatistics,38690905,Yan-Bo Zhang
3,Department of Epidemiology Biostatistics,38690905,Yi Wang
4,Department of Epidemiology Biostatistics,38690905,Tingting Geng
...,...,...,...
693,"Department of Experimental, Diagnostic Special...",24688116,Stefano Salvioli
694,Institute of Clinical Molecular Biology,24688116,Almut Nebel
695,"Epidemiology, Institute of Public Health Depar...",24688116,Lene Christiansen
696,Department of Medical Statistics Bioinformatics,24688116,Jeanine J Houwing-Duistermaat


In [16]:
df_count

,Affiliations,Country,PubMed ID,Author
0,"Department of Geriatrics, Institute of Geronto...",China,38853292,Yucong Zhang
1,"Key Laboratory of Vascular Aging, Ministry of ...",China,38853292,Yucong Zhang
2,"Department of Geriatrics, Institute of Geronto...",China,38853292,Man Liu
3,"Key Laboratory of Vascular Aging, Ministry of ...",China,38853292,Man Liu
4,"Division of Cardiothoracic Vascular Surgery, T...",China,38853292,Jiajun Li
...,...,...,...,...
178401,"St George's Hospital Medical School, St George...",UK,12848262,Shirley V Hodgson
178402,"The Lancet, 32 Jamestown Road, NW1 7BY, London...",UK,12767753,Virginia Barbour
178403,"Cellbank , Department of Genetics Mutagenesis,...",Japan,12693011,Tohru Masui
178404,"MRC Human Genetics Unit, Western General Hospi...",UK,12049178,A F Wright
